In [1]:
import numpy as np
import pandas as pd
import math
import scipy as sp
import statsmodels.discrete.discrete_model as sm_model
import statsmodels.tools as sm_tools
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.utils import resample

In [2]:
file1_path = "nswre74_treated.mat"
mat_data = sp.io.loadmat(file1_path)
mat_data.keys()

dict_keys(['__header__', '__version__', '__globals__', 't', 'age', 'education', 'black', 'hispanic', 'married', 'nodegree', 're74', 're75', 're78', 'u74', 'u75'])

In [4]:
# Extract the variables
treated = mat_data['t'].flatten()
age = mat_data['age'].flatten()
educ = mat_data['education'].flatten()
black = mat_data['black'].flatten()
hisp = mat_data['hispanic'].flatten()
married = mat_data['married'].flatten()
nodegree = mat_data['nodegree'].flatten()
earn_74 = mat_data['re74'].flatten()
earn_75 = mat_data['re75'].flatten()
earn_78 = mat_data['re78'].flatten()
u_74 = mat_data['u74'].flatten()
u_75 = mat_data['u75'].flatten()

# Create a DataFrame
df_treat = pd.DataFrame({
    'treated': treated,
    'age': age,
    'educ': educ,
    'black': black,
    'hisp': hisp,
    'married': married,
    'nodegree': nodegree,
    'earn_74': earn_74/1000,
    'earn_75': earn_75/1000,
    'earn_78': earn_78/1000,
    'u_74': u_74,
    'u_75': u_75
})
df_treat

,treated,age,educ,black,hisp,married,nodegree,earn_74,earn_75,earn_78,u_74,u_75
0,1,37,11,1,0,1,1,0.00000,0.00000,9.930046,1,1
1,1,22,9,0,1,0,1,0.00000,0.00000,3.595894,1,1
2,1,30,12,1,0,0,0,0.00000,0.00000,24.909450,1,1
3,1,27,11,1,0,0,1,0.00000,0.00000,7.506146,1,1
4,1,33,8,1,0,0,1,0.00000,0.00000,0.289790,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
180,1,33,12,1,0,1,0,20.27995,10.94135,15.952600,0,0
181,1,25,14,1,0,1,0,35.04007,11.53657,36.646950,0,0
182,1,35,9,1,0,1,1,13.60243,13.83064,12.803970,0,0
183,1,35,8,1,0,1,1,13.73207,17.97615,3.786628,0,0


In [5]:
file3_path = "cps_controls.mat"
mat_data = sp.io.loadmat(file3_path)
mat_data.keys()

dict_keys(['__header__', '__version__', '__globals__', 't', 'age', 'education', 'black', 'hispanic', 'married', 'nodegree', 're74', 're75', 're78', 'u74', 'u75'])

In [6]:
file3_path = "cps_controls.mat"
mat_data = sp.io.loadmat(file3_path)
mat_data.keys()

# Extract the variables
treated = mat_data['t'].flatten()
age = mat_data['age'].flatten()
educ = mat_data['education'].flatten()
black = mat_data['black'].flatten()
hisp = mat_data['hispanic'].flatten()
married = mat_data['married'].flatten()
nodegree = mat_data['nodegree'].flatten()
earn_74 = mat_data['re74'].flatten()
earn_75 = mat_data['re75'].flatten()
earn_78 = mat_data['re78'].flatten()
u_74 = mat_data['u74'].flatten()
u_75 = mat_data['u75'].flatten()

# Create a DataFrame
df_control = pd.DataFrame({
    'treated': treated,
    'age': age,
    'educ': educ,
    'black': black,
    'hisp': hisp,
    'married': married,
    'nodegree': nodegree,
    'earn_74': earn_74/1000,
    'earn_75': earn_75/1000,
    'earn_78': earn_78/1000,
    'u_74': u_74,
    'u_75': u_75,
})

1.a. Contruct table with normalized differences in covars

In [7]:
treat_covars = df_treat.iloc[:, 1:].drop(columns=['earn_78'])
control_covars = df_control.iloc[:, 1:].drop(columns=['earn_78'])

In [8]:
df_diff = treat_covars.mean()-control_covars.mean()
df_diff

age         -7.409021
educ        -1.681568
black        0.769706
hisp        -0.012577
married     -0.522542
nodegree     0.412273
earn_74    -11.921227
earn_75    -12.118748
u_74         0.588486
u_75         0.490695
dtype: float64

In [9]:
s_0 = (np.sum((control_covars - control_covars.mean())**2))/(len(control_covars)-1)
s_0

/Users/tmullane/Library/Python/3.9/lib/python/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


age         121.996792
educ          8.241754
black         0.068133
hisp          0.066851
married       0.205183
nodegree      0.208330
earn_74      91.580993
earn_75      85.940376
u_74          0.105319
u_75          0.097363
dtype: float64

In [10]:
s_1 = (np.sum((treat_covars - treat_covars.mean())**2))/(len(treat_covars)-1)
s_1

age         51.194301
educ         4.042714
black        0.132902
hisp         0.056228
married      0.154230
nodegree     0.207814
earn_74     23.879058
earn_75     10.363576
u_74         0.207814
u_75         0.241304
dtype: float64

In [11]:
norm_dist = df_diff/np.sqrt((s_0 + s_1)/2)
norm_dist

age        -0.796183
educ       -0.678502
black       2.427747
hisp       -0.050697
married    -1.232648
nodegree    0.903811
earn_74    -1.568990
earn_75    -1.746428
u_74        1.487257
u_75        1.192450
dtype: float64

b. Estimate propensity score (linear covars)

In [12]:
df = pd.concat([df_treat, df_control], axis=0, ignore_index=True)
df

,treated,age,educ,black,hisp,married,nodegree,earn_74,earn_75,earn_78,u_74,u_75
0,1,37,11,1,0,1,1,0.000000,0.000000,9.930046,1,1
1,1,22,9,0,1,0,1,0.000000,0.000000,3.595894,1,1
2,1,30,12,1,0,0,0,0.000000,0.000000,24.909450,1,1
3,1,27,11,1,0,0,1,0.000000,0.000000,7.506146,1,1
4,1,33,8,1,0,0,1,0.000000,0.000000,0.289790,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
16172,0,22,12,1,0,0,0,3.975352,6.801435,2.757438,0,0
16173,0,20,12,1,0,1,0,1.445939,11.832240,6.895072,0,0
16174,0,37,12,0,0,0,0,1.733951,1.559371,4.221865,0,0
16175,0,47,9,0,0,1,1,16.914350,11.384660,13.671930,0,0


In [13]:
covar_list = ['age', 'educ', 'black', 'hisp', 'married', 'nodegree', 'earn_74', 'earn_75', 'u_74', 'u_75']

In [14]:
covars = df[covar_list]
covars = sm_tools.add_constant(covars)
treat = df['treated']

In [15]:
# Formula-based approach
formula = "treated ~ " + " + ".join(covar_list)
probit_formula = smf.probit(formula, data=df)
probit_results = probit_formula.fit()
print(probit_results.summary())

Optimization terminated successfully.
         Current function value: 0.029675
         Iterations 11
                          Probit Regression Results                           
Dep. Variable:                treated   No. Observations:                16177
Model:                         Probit   Df Residuals:                    16166
Method:                           MLE   Df Model:                           10
Date:                Sun, 11 May 2025   Pseudo R-squ.:                  0.5252
Time:                        15:42:08   Log-Likelihood:                -480.05
converged:                       True   LL-Null:                       -1011.1
Covariance Type:            nonrobust   LLR p-value:                7.996e-222
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.9962      0.390     -7.681      0.000      -3.761      -2.232
age           -0.0103      0

In [16]:
# Calculate propensity scores
df['propensity_score'] = probit_results.predict()
# df['propensity_score'] = probit_results.predict(covars, linear=True)

# Examine the distribution of propensity scores
print(df['propensity_score'].describe())

count    1.617700e+04
mean     1.143629e-02
std      5.532375e-02
min      4.802004e-09
25%      1.232289e-06
50%      2.458182e-05
75%      9.665958e-04
max      5.605503e-01
Name: propensity_score, dtype: float64


c. Calculate weighted ATT

In [17]:
df['weight'] = np.where(
    df['treated'] == 1,
    1,
    df['propensity_score'] / (1 - df['propensity_score'])
)

In [18]:
weights_control = df['weight'][df['treated'] == 0]
weights_control_norm = weights_control * len(weights_control)/sum(weights_control)
print(weights_control_norm.describe())

count    1.599200e+04
mean     1.000000e+00
std      6.582768e+00
min      4.307648e-07
25%      1.076495e-04
50%      2.039980e-03
75%      7.395292e-02
max      1.144256e+02
Name: weight, dtype: float64


In [19]:
top_5_weights = weights_control_norm.sort_values(ascending=False).head(5)
top_5_weights

14156    114.425557
9569     114.425557
11452    114.425557
15757    113.556873
2761     113.421985
Name: weight, dtype: float64

In [20]:
covars.loc[top_5_weights.index]

,const,age,educ,black,hisp,married,nodegree,earn_74,earn_75,u_74,u_75
14156,1.0,16,10,1,0,0,1,0.0,0.0,1,1
9569,1.0,16,10,1,0,0,1,0.0,0.0,1,1
11452,1.0,16,10,1,0,0,1,0.0,0.0,1,1
15757,1.0,17,11,1,0,0,1,0.0,0.0,1,1
2761,1.0,16,9,1,0,0,1,0.0,0.0,1,1


In [21]:
def att(df):
    treated = df['treated'] == 1
    control = df['treated'] == 0
        
    weights_control = df['propensity_score'][control] / (1 - df['propensity_score'][control])
    weights_control_norm = weights_control * len(weights_control) / sum(weights_control)
        
    att = np.mean(df['earn_78'][treated]) - np.average(df['earn_78'][control], weights=weights_control_norm)
    return att

In [22]:
att_estimate = att(df)
att_estimate

np.float64(1.214120782392138)

In [23]:
att_boots = []
for _ in range(1000):
    boot_sample = resample(df)
    att_boots.append(att(boot_sample))

att_std_error = np.std(att_boots)
print("SE=", att_std_error)

SE= 0.6516848512315674


d. Repeat with logit

In [24]:
# Formula-based approach
formula = "treated ~ " + " + ".join(covar_list)
logit_formula = smf.logit(formula, data=df)
logit_results = logit_formula.fit()
print(logit_results.summary())

Optimization terminated successfully.
         Current function value: 0.029377
         Iterations 12
                           Logit Regression Results                           
Dep. Variable:                treated   No. Observations:                16177
Model:                          Logit   Df Residuals:                    16166
Method:                           MLE   Df Model:                           10
Date:                Sun, 11 May 2025   Pseudo R-squ.:                  0.5300
Time:                        15:42:24   Log-Likelihood:                -475.23
converged:                       True   LL-Null:                       -1011.1
Covariance Type:            nonrobust   LLR p-value:                6.679e-224
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.3408      0.812     -7.804      0.000      -7.933      -4.748
age           -0.0179      0

In [ ]:
# logit = sm_model.Logit(treat, covars)
# logit_results = logit.fit(maxiter=10000)
# print("logit:",logit_results.summary())

In [25]:
# Calculate propensity scores
# df['propensity_score'] = logit_results.predict(covars, linear=True)
df['propensity_score_log'] = logit_results.predict()

# Examine the distribution of propensity scores
print(df['propensity_score_log'].describe())

count    16177.000000
mean         0.011436
std          0.059658
min          0.000005
25%          0.000048
50%          0.000208
75%          0.001744
max          0.637511
Name: propensity_score_log, dtype: float64


In [26]:
df['weight_log'] = np.where(
    df['treated'] == 1,
    1,
    df['propensity_score_log'] / (1 - df['propensity_score_log'])    # equation when flag == 0
)

In [27]:
weights_control_log = df['weight_log'][df['treated'] == 0]
weights_control_norm_log = weights_control_log * len(weights_control_log)/sum(weights_control_log)
print(weights_control_norm_log.describe())

count    15992.000000
mean         1.000000
std          7.806084
min          0.000400
25%          0.004056
50%          0.016981
75%          0.134161
max        149.498966
Name: weight_log, dtype: float64


In [28]:
top_5_weights_log = weights_control_norm_log.sort_values(ascending=False).head(5)
top_5_weights_log

15757    149.498966
11452    149.262577
14156    149.262577
9569     149.262577
2781     146.846240
Name: weight_log, dtype: float64

In [29]:
covars.loc[top_5_weights_log.index]

,const,age,educ,black,hisp,married,nodegree,earn_74,earn_75,u_74,u_75
15757,1.0,17,11,1,0,0,1,0.0,0.0,1,1
11452,1.0,16,10,1,0,0,1,0.0,0.0,1,1
14156,1.0,16,10,1,0,0,1,0.0,0.0,1,1
9569,1.0,16,10,1,0,0,1,0.0,0.0,1,1
2781,1.0,18,11,1,0,0,1,0.0,0.0,1,1


In [30]:
att_estimate_logit = att(df)
att_estimate_logit

np.float64(1.214120782392138)

In [31]:
att_logit_boots = []
for _ in range(1000):
    boot_sample = resample(df)
    att_logit_boots.append(att(boot_sample))

att_logit_std_error = np.std(att_logit_boots)
print("SE=", att_logit_std_error)

SE= 0.6614136335714766


e. Construct strata

In [32]:
treated_ps = df[df['treated'] == 1]['propensity_score']
quintiles = np.percentile(treated_ps, [20, 40, 60, 80])
df['stratum'] = pd.cut(df['propensity_score'], 
                       bins=[-np.inf] + list(quintiles) + [np.inf], 
                       labels=[1, 2, 3, 4, 5])
control_counts = df[df['treated'] == 0]['stratum'].value_counts().sort_index()
control_counts

stratum
1    15639
2      230
3       56
4       41
5       26
Name: count, dtype: int64

f. Normalized differences in each stratum

In [33]:
s_0.index.values

array(['age', 'educ', 'black', 'hisp', 'married', 'nodegree', 'earn_74',
       'earn_75', 'u_74', 'u_75'], dtype=object)

In [34]:
s_0['age']

np.float64(121.99679157632723)

In [35]:
norm_dist = {}
for stratum in range(1, 6):
    strat_cov = {}
    stratum_data = df[df['stratum'] == stratum].drop(columns=['earn_78','propensity_score', 'stratum','weight', 'weight_log'])
    treat_covars = stratum_data[stratum_data['treated'] == 1].iloc[:, 1:]
    control_covars = stratum_data[stratum_data['treated'] == 0].iloc[:, 1:]
    df_diff = treat_covars.mean()-control_covars.mean()
    s_0 = (np.sum((control_covars - control_covars.mean())**2))/(len(control_covars)-1)
    s_1 = (np.sum((treat_covars - treat_covars.mean())**2))/(len(treat_covars)-1)

    for cov in s_0.index.values:
        if s_0[cov] == 0 and s_1[cov] == 0:
            strat_cov[cov] = 0
        elif s_0[cov] == 0 or s_1[cov] == 0:
            strat_cov[cov] = np.inf if df_diff[cov]!= 0 else 0
        else:
            strat_cov[cov] = df_diff[cov]/np.sqrt((s_0[cov] + s_1[cov])/2)

        norm_dist[stratum] = strat_cov

norm_dist

/Users/tmullane/Library/Python/3.9/lib/python/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/Users/tmullane/Library/Python/3.9/lib/python/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/Users/tmullane/Library/Python/3.9/lib/python/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pa

{1: {'age': np.float64(-0.8283937440307428),
  'educ': np.float64(-0.5207482128232125),
  'black': np.float64(0.7222275192544375),
  'hisp': np.float64(0.35354356160103456),
  'married': np.float64(-0.6108518144498548),
  'nodegree': np.float64(0.5832893526051861),
  'earn_74': np.float64(-1.079837104378044),
  'earn_75': np.float64(-1.2586369988054609),
  'u_74': np.float64(0.7725178375246264),
  'u_75': np.float64(0.5044070216857219),
  'propensity_score_log': np.float64(1.0966187685552686)},
 2: {'age': np.float64(-0.07860070754308357),
  'educ': np.float64(-0.2916891495235184),
  'black': np.float64(0.21113639817526061),
  'hisp': np.float64(-0.21113639817526078),
  'married': np.float64(-0.35917182631596795),
  'nodegree': np.float64(0.25379913431080653),
  'earn_74': np.float64(0.28419065886072353),
  'earn_75': np.float64(0.31075117848888645),
  'u_74': np.float64(-0.48137412518673117),
  'u_75': np.float64(-0.48137412518673117),
  'propensity_score_log': np.float64(0.0051801592

In [36]:
stratum_sizes = df[df['treated'] == 1]['stratum'].value_counts().sort_index()

In [37]:
weighted_diffs = {}
for cov in covars.columns[1:]:
    weighted_diff = sum(norm_dist[s][cov] * stratum_sizes[s] for s in range(1, 6)) / sum(stratum_sizes)
    weighted_diffs[cov] = weighted_diff

for cov, diff in weighted_diffs.items():
    print(cov, diff)

age 0.007111496327081107
educ -0.10150103653922386
black 0.18667278348593963
hisp 0.02848143268515476
married -0.16689814096693034
nodegree 0.13453389637769475
earn_74 -0.15661525321137995
earn_75 -0.1959210639204409
u_74 0.15060769393436815
u_75 0.15532775597568882


In [38]:
# Identify unbalanced covariates (e.g., |diff| > 0.1)
unbalanced = {cov: diff for cov, diff in weighted_diffs.items() if abs(diff) > 0.1}
if unbalanced:
    print("\nUnbalanced covariates:")
    for cov, diff in unbalanced.items():
        print(f"{cov}: {diff:.4f}")
else:
    print("\nNo covariates are very unbalanced.")

# Identify most imbalanced stratum
stratum_imbalance = {}
for stratum in range(1, 6):
    stratum_imbalance[stratum] = max(abs(diff) for diff in norm_dist[stratum].values())

most_imbalanced = max(stratum_imbalance, key=stratum_imbalance.get)
print(f"\nMost imbalanced stratum: {most_imbalanced} (max normalized difference: {stratum_imbalance[most_imbalanced]:.4f})")


Unbalanced covariates:
educ: -0.1015
black: 0.1867
married: -0.1669
nodegree: 0.1345
earn_74: -0.1566
earn_75: -0.1959
u_74: 0.1506
u_75: 0.1553

Most imbalanced stratum: 1 (max normalized difference: 1.2586)


g. Avg effect within each strata 

In [43]:
def lin_model(df):
    covars = df.drop(columns=['earn_78'])
    covars = sm_tools.add_constant(covars)
    out = df['earn_78']
    lin_reg = smf.ols("earn_78 ~ treated +" + " + ".join(covar_list), data=df).fit()
    # lin_reg = sm.OLS(out, covars).fit()
    return lin_reg.params['treated'], lin_reg.bse['treated'], lin_reg.pvalues['treated'], lin_reg

results = {}
for strata in range(1, 6):
    df_stat = df[df['stratum'] == strata].drop(columns=['propensity_score', 'stratum','weight', 'weight_log'])
    effect, se, p_value, model = lin_model(df_stat)
    results[strata]= {
            'effect': effect,
            'se': se,
            'p_value': p_value,
            'n_treated': sum(stratum_data['treated'] == 1),
            'n_control': sum(stratum_data['treated'] == 0),
            'model': model
        }
results

{1: {'effect': np.float64(-0.8307891513270041),
  'se': np.float64(1.1586019300602008),
  'p_value': np.float64(0.47334667501357064),
  'n_treated': 36,
  'n_control': 26,
  'model': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x1756746a0>},
 2: {'effect': np.float64(-1.070045728000303),
  'se': np.float64(1.0813285994599031),
  'p_value': np.float64(0.32332080346439673),
  'n_treated': 36,
  'n_control': 26,
  'model': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x175416430>},
 3: {'effect': np.float64(6.589213096504544),
  'se': np.float64(1.5874801706285606),
  'p_value': np.float64(7.982832271993558e-05),
  'n_treated': 36,
  'n_control': 26,
  'model': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x175423e20>},
 4: {'effect': np.float64(5.014111531001372),
  'se': np.float64(2.1200538508170195),
  'p_value': np.float64(0.020722942602797882),
  'n_treated': 36,
  'n_control': 26,
  'model': <statsmodels.regression.linear

In [44]:
# Calculate overall weighted average effect
total_treated = sum(res['n_treated'] for res in results.values())
weighted_effect = sum(res['effect'] * res['n_treated'] for res in results.values()) / total_treated
weighted_effect

np.float64(1.942896682474662)

h. ATT using within strata

In [45]:
def strata_att(df):
    strata_effect = {}
    strata_weight = {}

    for strata in range(1, 6):
        df_stat = df[df['stratum'] == strata].drop(columns=['propensity_score', 'stratum','weight', 'weight_log'])
        effect = results[strata]['effect']
        n_treated = results[strata]['n_treated']
        
        strata_effect[strata] = effect
        strata_weight[strata] = n_treated
    total_weight = sum(strata_weight.values())
    weighted_effect = sum(strata_effect[s] * strata_weight[s] for s in range(1, 6)) / total_weight

    return weighted_effect

In [46]:
att_strata = strata_att(df)

In [47]:
att_boots = []
for _ in range(1000):
    boot_sample = resample(df)
    att_boots.append(strata_att(boot_sample))

att_std_error = np.std(att_boots)
print("SE=", att_std_error)

SE= 4.440892098500626e-16


The standard error of the strata estimates is smaller than for the strictly weighted estimates. This is expected because by construction, although total N in the regression is less, the control group more closely resembles the treated group. This makes the variance of observables and outcomes smaller, leading to smaller SE. 

i. Are these estimates reasonable? 

These estimates both assume unconfoundedness, which may or may not be valid depending on whether you believe an argument about treatment being a related to the distrobutions of previous earnings or actions. Also, although the SE is smaller and the estimate is more precise for the stratified regression, this regression loses some of the nuance of the relationships of differences between groups as opposed to the more strictly defined differences within groups. 